In [1]:
import pandas as pd
from pymongo import MongoClient


def _connect_mongo(uri, db):
    """ A util for making a connection to mongo """

    if uri:
#         mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(uri)
    else:
        conn = MongoClient(host, port)
    return conn[db]

def read_mongo(uri, db, collection, query={}, no_id=True, limit=None):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    con = _connect_mongo(uri, db)

    # Make a query to the specific DB and Collection
    ### SET THE LIMIT TO 5 FOR DEV AND TESTING ###
    if limit:
        cursor = con[collection].find(query)
        cursor = cursor[limit]
    else:
        cursor = con[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df = pd.DataFrame(list(cursor))
    # Delete the _id
#     if no_id:
#         del df['_id']
    if limit:
        print(f'number of indexes created has been limited to {limit} ..........................')

    return df


In [2]:
import config
import overalls
import weather

In [3]:
w = weather.Weather('27606', 'observation')

In [4]:
print(w.weather)

{'_id': 'DEFAULT', 'clouds': 'DEFAULT', 'rain': {'1h': 0, '3h': 0}, 'snow': {'1h': 0, '3h': 0}, 'wind': {'speed': 0, 'deg': 0}, 'humidity': 'DEFAULT', 'pressure': {'press': 'DEFAULT', 'sea_level': 'DEFAULT'}, 'temperature': {'temp': 'DEFAULT', 'temp_max': 'DEFAULT', 'temp_min': 'DEFAULT'}, 'status': 'DEFAULT', 'detailed_status': 'DEFAULT', 'weather_code': 'DEFAULT', 'visibility_distance': 0, 'dewpoint': 'DEFAULT', 'humidex': 'DEFAULT', 'heat_index': 'DEFAULT', 'time_to_instant': 'DEFAULT'}


In [5]:
collection = 'legit_inst'
db = 'owmap'
df = read_mongo(config.uri, db, collection, no_id=False, limit=5)

number of indexes created has been limited to 5 ..........................


In [6]:
def read_mongo_a(uri, db, collection, limit=None):
    ''' Retrieve data from the Mongo database and transform it to a pandas
    DataFrame; return the DataFrame.

    :param collection: the collection name
    :type collection: string
    :param limit: optional limiter to the number of documents retrieved. '''
    
    database = _connect_mongo(config.uri, db)
    col = {}
    
    # Shorten the cursor length if limit is given, otherwise get everything;
    # transform the retrieved data to a pandas.DataFrame and return it.
    if limit:
        cursor = database[collection].find({})
        cursor = cursor
        for doc in cursor[:limit]:
            col[doc['_id']] = doc
        # Transform the data collection to a dataframe.
        df = pd.DataFrame.from_dict(col, orient='index')
        print(f'The length of your df has been limited to {limit}.')
        return df
    else:
        cursor = database[collection].find({})
        for doc in cursor:
            col[doc['_id']] = doc
        # Transform the data collection to a dataframe.
        df = pd.DataFrame.from_dict(col, orient='index')
        return df
    
    # Let the user know that even though there were no errors the DataFrame
    # was not created.
    print('From read_mongo(): There were no errors, but your dataframe was not created.')


# def all_values(d, values=[]):
#     ''' Get all the dicitonary and nested dictionary keys from a dict, add them
#     to a list, return the list.
    
#     :param d: A python dictionary
#     :return: A list of every key in the dictionary
#     '''

#     for key, value in d.items():
#         if isinstance(d[key], dict):
#             all_values(d[key], values=values)
#         else:
#             values.append(value)
#     return values


In [7]:
df = read_mongo_a(config.uri, db, collection, limit=5)
df.set_index(['_id'])

The length of your df has been limited to 5.


,instant,zipcode,forecasts,weather
_id,,,,
5eba42b80923c58fcff13738,1589695200,27152,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674843, 'sunrise_time': 15..."
5eba42b80923c58fcff13739,1589695200,27155,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674843, 'sunrise_time': 15..."
5eba42b80923c58fcff1373a,1589695200,27157,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674843, 'sunrise_time': 15..."
5eba42b80923c58fcff1373b,1589695200,27198,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674843, 'sunrise_time': 15..."
5eba42b80923c58fcff1373c,1589695200,27199,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674860, 'sunrise_time': 15..."


In [8]:
obs = df['weather'][4]
# print(df['weather'][4])
# overalls.flatten(obs)
errs = overalls.compare_dicts(obs, df['weather'][3])
# print(overalls.flatten_dict(errs), '\n')
W = weather.Weather('27606', 'observation', data=errs)

{'sunset_time': 1589674860, 'sunrise_time': 1589624034, 'clouds': 1, 'rain': {}, 'snow': {}, 'wind': {'speed': 2.59, 'deg': 54}, 'humidity': 77, 'pressure': {'press': 1018, 'sea_level': None}, 'temperature': {'temp': 292.3, 'temp_kf': None, 'temp_max': 294.26, 'temp_min': 290.93}, 'status': 'Clear', 'detailed_status': 'clear sky', 'weather_code': 800, 'weather_icon_name': '01n', 'visibility_distance': 16093, 'dewpoint': None, 'humidex': None, 'heat_index': None, 'time_to_instant': 8981}


In [9]:
W.weather

{'_id': 'DEFAULT',
 'clouds': 0,
 'rain': {},
 'snow': {},
 'wind': {'speed': 0.41999999999999993, 'deg': -12},
 'humidity': 0,
 'pressure': {'press': 0},
 'temperature': {'temp': 0.29000000000002046,
  'temp_max': 0.0,
  'temp_min': 0.5600000000000023},
 'status': 0,
 'detailed_status': 0,
 'weather_code': 0,
 'visibility_distance': 0,
 'dewpoint': 'DEFAULT',
 'humidex': 'DEFAULT',
 'heat_index': 'DEFAULT',
 'time_to_instant': 44,
 'sunset_time': 17,
 'sunrise_time': -2,
 'weather_icon_name': 0}

In [7]:
data = overalls.flatten_dict(errs)
print(data)
d = {'clouds':10}
data.update(d)
print(data)

{'sunset_time': 17, 'sunrise_time': -2, 'clouds': 0, 'wind.speed': 0.41999999999999993, 'wind.deg': -12, 'humidity': 0, 'pressure.press': 0, 'temperature.temp': 0.29000000000002046, 'temperature.temp_max': 0.0, 'temperature.temp_min': 0.5600000000000023, 'status': 0, 'detailed_status': 0, 'weather_code': 0, 'weather_icon_name': 0, 'visibility_distance': 0, 'time_to_instant': 44}
{'sunset_time': 17, 'sunrise_time': -2, 'clouds': 10, 'wind.speed': 0.41999999999999993, 'wind.deg': -12, 'humidity': 0, 'pressure.press': 0, 'temperature.temp': 0.29000000000002046, 'temperature.temp_max': 0.0, 'temperature.temp_min': 0.5600000000000023, 'status': 0, 'detailed_status': 0, 'weather_code': 0, 'weather_icon_name': 0, 'visibility_distance': 0, 'time_to_instant': 44}


In [8]:
errors = []
for (obs, casts) in zip(df['weather'], df['forecasts']):
    error = [overalls.compare_dicts(cast, obs) for cast in casts]
    errors.append(error)
df['errors'] = errors
len(errors)

AttributeError: 'str' object has no attribute 'keys'

In [10]:
errors[0]

[{'sunset_time': -1589674843,
  'sunrise_time': -1589624036,
  'clouds': -1,
  'humidity': 7,
  'status': 0,
  'detailed_status': 0,
  'weather_code': 0,
  'weather_icon_name': 0,
  'time_to_instant': 421156},
 {'sunset_time': -1589674843,
  'sunrise_time': -1589624036,
  'clouds': 26,
  'humidity': 5,
  'status': 1,
  'detailed_status': 1,
  'weather_code': 2,
  'weather_icon_name': 1,
  'time_to_instant': 410357},
 {'sunset_time': -1589674843,
  'sunrise_time': -1589624036,
  'clouds': 26,
  'humidity': 5,
  'status': 1,
  'detailed_status': 1,
  'weather_code': 2,
  'weather_icon_name': 1,
  'time_to_instant': 399563},
 {'sunset_time': -1589674843,
  'sunrise_time': -1589624036,
  'clouds': -1,
  'humidity': 3,
  'status': 0,
  'detailed_status': 0,
  'weather_code': 0,
  'weather_icon_name': 0,
  'time_to_instant': 388757},
 {'sunset_time': -1589674843,
  'sunrise_time': -1589624036,
  'clouds': -1,
  'humidity': 3,
  'status': 0,
  'detailed_status': 0,
  'weather_code': 0,
  'wea

In [10]:
df['errors'] = errors
df.head()

,_id,instant,zipcode,forecasts,weather,errors
5eba42b80923c58fcff13738,5eba42b80923c58fcff13738,1589695200,27152,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674843, 'sunrise_time': 15...","[{'sunset_time': -1589674843, 'sunrise_time': ..."
5eba42b80923c58fcff13739,5eba42b80923c58fcff13739,1589695200,27155,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674843, 'sunrise_time': 15...","[{'sunset_time': -1589674843, 'sunrise_time': ..."
5eba42b80923c58fcff1373a,5eba42b80923c58fcff1373a,1589695200,27157,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674843, 'sunrise_time': 15...","[{'sunset_time': -1589674843, 'sunrise_time': ..."
5eba42b80923c58fcff1373b,5eba42b80923c58fcff1373b,1589695200,27198,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674843, 'sunrise_time': 15...","[{'sunset_time': -1589674843, 'sunrise_time': ..."
5eba42b80923c58fcff1373c,5eba42b80923c58fcff1373c,1589695200,27199,"[{'sunset_time': 0, 'sunrise_time': 0, 'clouds...","{'sunset_time': 1589674860, 'sunrise_time': 15...","[{'sunset_time': -1589674860, 'sunrise_time': ..."
